# EDA 2 - Time of Read Data

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import os
import time
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from PIL import Image

import sys
sys.path.append(os.path.abspath('..'))

from tools.data_reader import DataReader
from tools.decode_raw import decode_img_seg

In [2]:
dr = DataReader()

read_train_params = {
    'path': '../data/train.csv',
    'train_path': '../data/train_images',
    'height': 256,
    'width': 1600, 
    'col': False,
    'sep':'[_,]'
}

gen = dr.read_train(**read_train_params)
count = dr.count('../data/train_images')

In [ ]:
for i in tqdm(gen):
    pass

In [ ]:
feature_dict = {'img': 'bytes', 'label': 'bytes'}

write_tfr_params = {
    'data_generator': gen,
    'count': count,
    'tfrpath': 'train_1',
    'feature_dict': feature_dict,
    'shards': 10,
    'compression': 'GZIP', 'c_level': None}

dr.write_tfr(**write_tfr_params)

In [ ]:
files = tf.train.match_filenames_once(os.path.join('..', 'tmp', 'TFRecords', 'train', '*'))


with tf.Session() as sess:
    # tf.local_variables_initializer().run()
    sess.run(tf.local_variables_initializer())
    print(sess.run(files))

In [ ]:
read_params = {
    'feature_dict': {'img': 'bytes', 'label': 'bytes'},
    'tfr_path': os.path.join('..', 'tmp', 'TFRecords', 'train_1', '*'),
    'decode_raw': decode_img_seg,
    'shuffle_buffer': 1000,
    'num_valid': 2000,
    'compression': 'GZIP'}

dr = DataReader()
dr.readtfrecorde(**read_params)

In [7]:
def parse(raw):
    features = {'img': tf.io.FixedLenFeature([], tf.string), 'label': tf.io.FixedLenFeature([], tf.strings)}
    raw = tf.parse_single_example(raw, features=features)
    return raw 

tfr_path = os.path.join('..', 'tmp', 'TFRecords', 'train_1', '*')
files = tf.io.match_filenames_once(tfr_path)
dataset = tf.data.TFRecordDataset(files)
dataset = dataset.map(parse)

TypeError: Cannot convert value <module 'tensorflow._api.v1.strings' from '/home/leechh/anaconda3/envs/severstal/lib/python3.7/site-packages/tensorflow/_api/v1/strings/__init__.py'> to a TensorFlow DType.